### 先都加载到cpu上

In [3]:
# pip install git+https://github.com/mobiusml/hqq.git;
# pip install bitblas #to use the bitblas backend
# OMP_NUM_THREADS=16 CUDA_VISIBLE_DEVICES=0 ipython3 
########################################################################
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import torch
device        = 'cuda:0'
backend       = 'torchao_int4' #"torchao_int4" (4-bit only) or "bitblas" (4-bit + 2-bit)
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
cache_dir     = '.' 
model_id      = '/home/bcds/venv/dilab/Meta-Llama-3.1-8B'

########################################################################
#Load model
from transformers import AutoModelForCausalLM, AutoTokenizer, HqqConfig 

quant_config = HqqConfig(nbits=4, group_size=64, axis=1)

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=compute_dtype, 
    cache_dir=cache_dir,
    device_map=device, 
    low_cpu_mem_usage=True,
    quantization_config=quant_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

#Patching
from hqq.utils.patching import prepare_for_inference
prepare_for_inference(model, backend=backend, verbose=True) 

########################################################################
#Inference
from hqq.utils.generation_hf import patch_model_for_compiled_runtime
patch_model_for_compiled_runtime(model, tokenizer, warmup=True) 

prompt = "Write an essay about large language models."

inputs = tokenizer([tokenizer.apply_chat_template([{"role": "user", "content": prompt},], tokenize=False)], return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000, cache_implementation="static", pad_token_id=tokenizer.pad_token_id) 
#print(tokenizer.decode(outputs[0]))

  0%|          | 0/225 [00:00<?, ?it/s]


RuntimeError: _convert_weight_to_int4pack_cuda is not available for build.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ["TOKENIZERS_PARALLELISM"] = "False"
import torch
device        = 'cuda:0'
backend       = "torchao_int4" #'torchao_int4' #"torchao_int4" (4-bit only) or "bitblas" (4-bit + 2-bit) or "gemlite" (8-bit, 4-bit, 2-bit, 1-bit)
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
cache_dir     = '.' 
model_id      = '/home/bcds/venv/dilab/Meta-Llama-3.1-8B'

########################################################################
#Load model
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *

#Load
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
model     = LlamaForCausalLM.from_pretrained(model_id, cache_dir=cache_dir, torch_dtype=compute_dtype, attn_implementation="sdpa")

#Quantize
quant_config = BaseQuantizeConfig(nbits=4, group_size=64, axis=1)
AutoHQQHFModel.quantize_model(model, quant_config=quant_config, compute_dtype=compute_dtype, device=device)
HQQLinear.set_backend(HQQBackend.PYTORCH)

#Optimize
from hqq.utils.patching import prepare_for_inference
prepare_for_inference(model, backend=backend, verbose=True)

#Load GemLite cache
if(backend == 'gemlite'):
	import gemlite
	gemlite.core.GEMLITE_TRITON_RESTRICT_M = True
	gemlite.core.GemLiteLinear.load_config('/tmp/gemlite_config.json')


/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/225 [00:00<?, ?it/s]


RuntimeError: _convert_weight_to_int4pack_cuda is not available for build.

In [2]:
chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ 'User: ' + message['content'] }}{% elif message['role'] == 'assistant' %}{{ 'Assistant: ' + message['content'] }}{% else %}{{ message['content'] }}{% endif %}{% if not loop.last %}\n{% endif %}{% endfor %}"
tokenizer.chat_template = chat_template
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

You are a friendly chatbot who always responds in the style of a pirateUser: How many helicopters can a human eat in one sitting?


In [3]:
#Inference
########################################################################
# #Using a custom generator
# from hqq.utils.generation_hf import HFGenerator
# gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile="partial").warmup() 

# out = gen.generate("Write an essay about large language models.", print_tokens=True)
########################################################################
#Using HF model.generate()
from hqq.utils.generation_hf import patch_model_for_compiled_runtime

patch_model_for_compiled_runtime(model, tokenizer, warmup=True)

  0%|          | 0/5 [00:00<?, ?it/s]The 'batch_size' attribute of StaticCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.
Using `torch.compile`.
  0%|          | 0/5 [02:20<?, ?it/s]


Unsupported: 'skip function sdpa_kernel in file /home/bcds/.conda/envs/llm/lib/python3.9/contextlib.py'

from user code:
   File "/home/bcds/venv/dilab/floe/hqq/hqq/utils/generation_hf.py", line 80, in custom_forward
    with sdpa_kernel([SDPBackend.MATH]):

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [5]:
# Prompt
system_prompt = None #"You are a helpful assistant."
prompt        = "Write an essay about large language models."

messages  = [] if(system_prompt is None) else [{"role": "system", "content": system_prompt}]
messages += [{"role": "user", "content": prompt},]

inputs = tokenizer([tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)],return_tensors="pt").to(model.device)

# Generate
outputs = model.generate(**inputs, max_new_tokens=32, cache_implementation="static", pad_token_id=tokenizer.pad_token_id) 
print(tokenizer.decode(outputs[0]))

########################################################################
#Save gemlite cache
if(backend == 'gemlite'):
	gemlite.core.GemLiteLinear.cache_config('/tmp/gemlite_config.json') 

  0%|          | 0/5 [03:55<?, ?it/s]


Unsupported: 'skip function sdpa_kernel in file /home/bcds/.conda/envs/llm/lib/python3.9/contextlib.py'

from user code:
   File "/home/bcds/venv/dilab/floe/hqq/hqq/utils/generation_hf.py", line 80, in custom_forward
    with sdpa_kernel([SDPBackend.MATH]):

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [2]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 1
MAX_LENGTH = input_length
output_length = 32
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time, prefill_time = 0, 0, 0
# print("max output length is {}".format(output_length))
text = "The future of AI is here, and "

inputs = preprocess_data(text, tokenizer)

# 前向传播
for i in range(2):
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(device_id),
            attention_mask=inputs["attention_mask"].cuda(device_id),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.pad_token_id, 
            # cache_implementation="static" ## moe not support
        )

/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/torch/cuda/graphs.py:84: UserWarning: The CUDA Graph is empty. This usually means that the graph was attempted to be captured on wrong device or stream. (Triggered internally at ../aten/src/ATen/cuda/CUDAGraph.cpp:208.)
  super().capture_end()


#### 测试一个正常输出

In [3]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 4
MAX_LENGTH = input_length
output_length = 10
device_id = 0
test_samples = 1

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

generated_all, decode_time, prefill_time = 0, 0, 0
# print("max output length is {}".format(output_length))
text = "The future of AI is here, and "

inputs = preprocess_data(text, tokenizer)
# # 测试时间
# start_event = torch.cuda.Event(enable_timing=True)
# end_event = torch.cuda.Event(enable_timing=True)

# 开始计时
# torch.cuda.synchronize()
# start_event.record()

# 前向传播
with torch.no_grad():
    output = llm.generate(
        input_ids=inputs["input_ids"].cuda(device_id),
        attention_mask=inputs["attention_mask"].cuda(device_id),
        max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
        generation_config=GenerationConfig(do_sample=False),
        pad_token_id=tokenizer.pad_token_id, 
        # cache_implementation="static" ## moe not support
    )

# 结束计时
# end_event.record()
# torch.cuda.synchronize()

# 计算时间
# elapsed_time = start_event.elapsed_time(end_event) / 1000  # 转换为秒
# decode_time += elapsed_time
# print(f"Generated length: {len(output[0]) - input_length}", f"Time taken: {elapsed_time:.2f} s")
# print(output)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

# generated_all += (len(output[0]) - input_length -1)

# timepertoken = (decode_time) / (generated_all)
# print("decode phase speed:", '{:.4f}'.format(1/timepertoken) , ' token/s')

['The 2019-20 season will']


In [5]:
import json
from datasets import load_dataset, Dataset
from transformers import GenerationConfig

input_length = 1
MAX_LENGTH = input_length
output_length = 2
test_samples = 4

with open("../path.json", "r") as f:
    paths = json.load(f)
    fineweb_path = paths["fineweb"]

def preprocess_data(data, tokenizer):
	# 使用 tokenizer 将文本数据转换为模型输入
	inputs = tokenizer(data, padding="max_length", truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
	inputs["labels"] = inputs.input_ids.clone()
	return inputs

fineweb = load_dataset("parquet",data_files=fineweb_path) #726000
fineweb_text = fineweb['train']['text'][:test_samples] 

print("output length is {}".format(output_length))
text = fineweb_text[0]
inputs = preprocess_data(text, tokenizer)

# cached_mlp.clear_load_from_cpu_stats()
with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ]
) as p:
    # 前向传播
    with torch.no_grad():
        output = llm.generate(
            input_ids=inputs["input_ids"].cuda(),
            attention_mask=inputs["attention_mask"].cuda(),
            max_length=input_length + output_length,  # 总长度为输入长度 + 输出长度
            generation_config=GenerationConfig(do_sample=False),
            pad_token_id=tokenizer.eos_token_id
        )
print(p.key_averages().table(
    sort_by="self_cpu_time_total", row_limit=-1))
p.export_chrome_trace("./compile.json")

output length is 2


/home/bcds/.conda/envs/llm/lib/python3.9/site-packages/torch/cuda/graphs.py:84: UserWarning: The CUDA Graph is empty. This usually means that the graph was attempted to be captured on wrong device or stream. (Triggered internally at ../aten/src/ATen/cuda/CUDAGraph.cpp:208.)
  super().capture_end()


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  Torch-Compiled Region        77.45%        2.833s        78.41%        2.869s       5.603ms       3.038ms         5.41%      16.160ms      31.563us           512  
                       Scheduler.codegen (dynamo_timed)         9.00%     329.124ms         9.00%     329.124ms     164.562ms       0.000us         0.00%       0.000us       0.000us             2  
         